In [1]:
using Altro
using RobotZoo
using StaticArrays
using LinearAlgebra
using TrajectoryOptimization
const TO = TrajectoryOptimization
using BenchmarkTools
using Plots
using PlotlyJS
using ForwardDiff, FiniteDiff
using RobotDynamics
const RD = RobotDynamics

RD.@autodiff struct MyBicycleModel <: RD.ContinuousDynamics
  ref::Symbol
  L::Float64
  lr::Float64
  function MyBicycleModel(; ref::Symbol=:rear, L::Real=2.7, lr::Real=1.5)
    @assert ref ∈ (:cg, :front, :rear)
    @assert L > 0 "($L) must be greater than 0"
    @assert L > lr "($L) must be greater than ($lr)"
    new(ref, L, lr)
  end
end

body = quote
  da = u[1]
  ϕ = u[2]

  θ = x[3]
  δ = x[4]
  v = x[5]
  a = x[6]
  if model.ref == :cg
    β = atan(model.lr * δ, model.L)
    s, c = sincos(θ + β)
    ω = v * cos(β) * tan(δ) / model.L
  elseif model.ref == :rear
    s, c = sincos(θ)
    ω = v * tan(δ) / model.L
  elseif model.ref == :front
    s, c = sincos(θ + δ)
    ω = v * sin(δ) / model.L
  end
  ẋ = v * c
  ẏ = v * s
end

@eval function RD.dynamics(model::MyBicycleModel, x, u)
  $body
  return SA[ẋ, ẏ, ω, ϕ, a, da]
end
@eval function RD.dynamics!(model::MyBicycleModel, xdot, x, u)
  $body
  xdot[1] = ẋ
  xdot[2] = ẏ
  xdot[3] = ω
  xdot[4] = ϕ
  xdot[5] = a
  xdot[6] = da
  return nothing
end
RD.state_dim(::MyBicycleModel) = 6
RD.control_dim(::MyBicycleModel) = 2

function BicycleCar(scenario=:parallel_park, ; N=51, x0)
  model = MyBicycleModel(; ref=:rear)
  n, m = size(model)

  opts = SolverOptions(penalty_initial=1e4, verbose=0,
    cost_tolerance_intermediate=1e-1)

  tf = 5.0
  dt = tf / (N - 1)

  xf = SA[13, -1.2, deg2rad(0), 0, 2.0, 0]

  # x, y, theta, delta, v, a
  Q = Diagonal(SA[10, 10, 60, 1, 1, 1])
  # jerk, phi
  ρ = 1.0
  R = ρ * Diagonal(SA[1, 1])
  Qf = Diagonal(SA_F64[10, 10, 60, 1, 1, 1])
  obj = LQRObjective(Q * dt, R * dt, Qf, xf, N)

  cons = ConstraintList(n, m, N)
  bnd_x_l = [-1, -2.4, Inf, -deg2rad(45), 0.0, -3]
  bnd_x_u = [20, 2.4, Inf, deg2rad(45), 6.0, 3]
  bnd_u_l = [-3, -deg2rad(45)]
  bnd_u_u = [3, deg2rad(45)]

  bnd = BoundConstraint(n, m, x_min=bnd_x_l, x_max=bnd_x_u, u_min=bnd_u_l,
    u_max=bnd_u_u)

  p = 3
  A = zeros(p, m + n)
  A[1, 5] = 1.0
  A[2, 5] = -1.0
  A[3, 2] = -1.0
  A = SMatrix{p,m + n,Float64}(A)
  b = zeros(p)
  b[1] = 4.0
  b[2] = -1.0
  b[3] = 1.3
  b = SVector{p,Float64}(b)
  lin = LinearConstraint(n, m, A, b, Inequality())

  goal = GoalConstraint(xf)

  add_constraint!(cons, bnd, 1:N-1)
  # add_constraint!(cons, goal, N)
  add_constraint!(cons, lin, 2:N)

  prob = Problem(model, obj, x0, tf, xf=xf, constraints=cons)
  initial_controls!(prob, SA[0.0, 0.0])
  rollout!(prob)


  return prob, opts
end


WebIO._IJuliaInit()

BicycleCar (generic function with 2 methods)

In [2]:
function loop(x0::Vector{Float64})
# x0 = SA_F64[0, 0, 0, 0, 4, 0]
x00 = SVector{length(x0)}(x0)
@show x00
bicycle = BicycleCar(:parallel_park, x0=x00)
solver = ALTROSolver(bicycle...)
solve!(solver)

X = states(solver)
U = controls(solver)
@show size(X)
@show size(U)
Plots.plot([x[1] for x in X], [x[2] for x in X])
X[2]
end

loop (generic function with 1 method)

In [3]:
x0 = Vector{Float64}([0, 0, 0, 0, 4, 0])

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 4.0
 0.0

In [4]:
x0 = Vector(loop(x0))

x00 = [0.0, 0.0, 0.0, 0.0, 4.0, 0.0]

SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 21
    Solve Time: 2369.114096 (ms)

  Covergence
    Terminal Cost: 997.8883235415047
    Terminal dJ: -0.0013891946807689237
    Terminal gradient: 0.0020920871458031663
    Terminal constraint violation: 5.8036411232365026e-8
    Solve Status: SOLVE_SUCCEEDED
size(X) = (51,)
size(U) = (50,)


6-element Vector{Float64}:
  0.39999830559701893
 -0.0007760981272082895
 -0.00582375640097174
 -0.07853981633761649
  3.999999992198381
 -1.5597688115208917e-7